In [18]:
player_mode = ["a", "b", "c"]
pawns_mode = [mode for mode in player_mode for _ in range(4)]

pawns_mode

['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c', 'c', 'c', 'c']

In [10]:
num_player = 4
pawn = [F"p{i}_pawn{j}" for i in range(num_player) for j in range(4)]
pawn

['p0_pawn0',
 'p0_pawn1',
 'p0_pawn2',
 'p0_pawn3',
 'p1_pawn0',
 'p1_pawn1',
 'p1_pawn2',
 'p1_pawn3',
 'p2_pawn0',
 'p2_pawn1',
 'p2_pawn2',
 'p2_pawn3',
 'p3_pawn0',
 'p3_pawn1',
 'p3_pawn2',
 'p3_pawn3']

In [31]:
import pandas as pd
import numpy as np

class SmartPawn:
    aggressive = {}
    kind = {}
    Strategy = {}
    def __init__(self, player_mode):
        self.num_player = len(player_mode)
        pawns = [F"P{i}_pawn{j}" for i in range(self.num_player) for j in range(4)]
        pawns_mode = [mode for mode in player_mode for _ in range(4)]
        initial_loc = ["not in game" for _ in range(0, self.num_player*4)]
        inital_pos = ["not in game" for _ in range(0, self.num_player*4)]
        self.main_df = pd.DataFrame({
            "pawn_id": pawns,
            "mode": pawns_mode,
            "loc": initial_loc,
            "position": inital_pos
        })
test = SmartPawn(player_mode=["Strategy", "kind", "aggressive"])
test.main_df

,pawn_id,mode,loc,position
0,P0_pawn0,Strategy,not in game,not in game
1,P0_pawn1,Strategy,not in game,not in game
2,P0_pawn2,Strategy,not in game,not in game
3,P0_pawn3,Strategy,not in game,not in game
4,P1_pawn0,kind,not in game,not in game
5,P1_pawn1,kind,not in game,not in game
6,P1_pawn2,kind,not in game,not in game
7,P1_pawn3,kind,not in game,not in game
8,P2_pawn0,aggressive,not in game,not in game
9,P2_pawn1,aggressive,not in game,not in game


In [ ]:
import pandas as pd
import numpy as np

class SmartPawn:
    aggressive = {}
    kind = {}
    Strategy = {}
    def __init__(self, player_mode):
        self.num_player = len(player_mode)
        pawns = [F"P{i}_pawn{j}" for i in range(self.num_player) for j in range(4)]
        pawns_mode = [mode for mode in player_mode for _ in range(4)]
        initial_loc = ["not in game" for _ in range(0, self.num_player*4)]
        inital_pos = ["not in game" for _ in range(0, self.num_player*4)]
        self.main_df = pd.DataFrame({
            "pawn_id": pawns,
            "mode": pawns_mode,
            "loc": initial_loc,
            "position": inital_pos
        })
    
    def move(self, player, rolled_number):
        target_pawns_df = self.main_df[~self.main_df['pawn_id'].str.startswith(F"P{player}_")]
        loc_target_pawns = target_pawns_df["loc"].values

        can_hit = [self.__can_hit(F"P{player}_pawn{i}", rolled_number, loc_target_pawns) for i in range(4)]

        if rolled_number == 6:
            can_come_in_game = [self.__can_come_in_game(F"P{player}_pawn{i}", rolled_number) for i in range(4)]
        else:
            can_come_in_game = [0 for _ in range(4)]
        print(can_hit)

    # private:
    def __can_hit(self, pawn_id, rolled_number, loc_target_pawns):
        loc_pawn = self.main_df.loc[self.main_df["pawn_id"] == pawn_id, "loc"].values[0]
        if loc_pawn in ["not in game", "finish game"]:
            return 0
        
        loc_pawn += rolled_number
        if loc_pawn > 47:
            loc_pawn = loc_pawn - 47
            # 47, Number of houses in the game
        
        if loc_pawn in loc_target_pawns:
            return 1
        else:
            return 0
        
    def __can_come_in_game(self, pawn_id, rolled_number):
        pos_pawn = self.main_df.loc[self.main_df["pawn_id"] == pawn_id, "position"].value[0]

        if pos_pawn == "not in game":
            return 1
        else:
            return 0
        


test = SmartPawn(player_mode=["Strategy", "kind", "aggressive"])
test.move(1, 5)

['not_in_game' 'not_in_game' 'not_in_game' 'not_in_game' 'not_in_game'
 'not_in_game' 'not_in_game' 'not_in_game']
[0, 0, 0, 0]
